# Assignment 1: Dino Fun World

### Assignment Description

You, in your role as a data explorer and visualizer, have been asked by the administrators of a small amusement park in your hometown to answer a few questions about their park operations. The dataset that they provided for you to perform the requested analysis includes the movement and communication data captured from the park attendees' apps during one weekend (Friday, Saturday, and Sunday).

The administrators would like you to answer four relatively simple questions about the park activities on the day in question. These questions all deal with park operations and can be answered using the data provided.
* **Question 1:** What is the most popular attraction to visit in the park?


* **Question 2:** What ride (note that not all attractions are rides) has the second longest average visit time?


* **Question 3:** Which Fast Food offering has the fewest visitors?


* **Question 4:** Compute the Skyline of number of visits and visit time for the park's ride, and report the rides that appear in the Skyline. (Note: Your answer should be three points, which can be given in any order.)


### Directions

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkin', 'attractions', and 'sequences'. The database file is named 'dinofunworld.db' .

The information contained in each of these tables is listed below:

`checkin`:
    - The check-in data for all visitors for the day in the park. The data includes two types of check-ins: inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type

`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carousel, etc.
    - Fields: AttractionID, Name, Region, Category, type
    
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
Using the provided data, answer the four questions that the administrators have asked.

### Submission Directions for Assignment Deliverables

This assignment will be auto-graded. We recommend that you submit this assignment. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly. 

Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed. 

Please execute each cell in Jupyter Notebook before submitting.




### Question 1: 
What is the most popular attraction to visit in the park?

**Note:** Your output should be the name of the attraction.

In [12]:
#importing sqlite to access the database through queries
import sqlite3

from datetime import timedelta

import math
#connecting to data base and setting the cursor to the database
data = sqlite3.connect('dinofunworld.db')
pointer = data.cursor()


In [13]:
# your code here for Q1

#from checkin table selecting attraction and concluding their count by putting on as ascending --> retriveing the last attraction
pointer.execute("SELECT attraction, COUNT(*) as cnt FROM checkin GROUP BY attraction ORDER BY cnt asc;")
#saving the ID of popular attraction
ID = pointer.fetchall()[-1][0]
#as ID is common key between checkin and attraction -->getting the name of attraction using ID on attraction table
pointer.execute("SELECT Name FROM attraction WHERE AttractionID="+str(ID)+";")
q1_ans = pointer.fetchone()[0]
print(q1_ans)

Atmosfear


### Question 2: 

What ride (note that not all attractions are rides) has the  longest average visit time?

**Note:** Your output should be the name of the ride.


In [14]:
# your code here for Q2
#filtering the ones with ride in the name using like operator and saving the results in list1
pointer.execute("SELECT AttractionID, Name FROM attraction where LOWER(Category) LIKE '%ride%';")
list1 = pointer.fetchall()
ride_time = ("", 0)
for (attraction_id, name) in list1:
    #getting duration from checkin table where attraction is same as attraction_id and saving them as list2
    pointer.execute("SELECT duration FROM checkin where attraction="+str(attraction_id)+";")
    list2 = pointer.fetchall()
    totalduration = 0
    i = 0
    for d in list2:
        try:
            duration = d[0].split(':')
            totalduration += (int(duration[0])*3600) + (int(duration[1])*60) +(int(duration[2]))
            i += 1
        except:
            continue
    average_duration = totalduration / i
    if average_duration > ride_time[1]:
        ride_time = (name, average_duration)
print(ride_time[0])

TerrorSaur


### Question 3:

Which Fast Food offering in the park has the fewest visitors?

**Note:** Your output should be the name of the fast food offering.

In [15]:
# your code here for Q3

#getting attraction and count by grouping by attraction from the table where attraction has fastfood in it
pointer.execute("SELECT attraction, COUNT(*) as cnt FROM checkin WHERE attraction IN (SELECT AttractionID FROM attraction where LOWER(type) LIKE '%fast food%') GROUP BY attraction ORDER by cnt asc;")
ID = pointer.fetchall()[0][0]
#get attraction name using the ID
pointer.execute("SELECT Name FROM attraction where AttractionID = " + str(ID) + ";")
q3_ans = pointer.fetchone()[0]
print(q3_ans)

Theresaur Food Stop


### Question 4:

Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline. 

**Note:** Remember that in this case, higher visits is better and lower visit times are better. Your output should be formatted as a list of names of the rides in the Skyline. Your output should be three points, which can be given in any order(example output : ['Ride 1','Ride 2','Ride 3'] )


In [16]:
#getting attraction and counting them as cnt from attractions where attraction id has ride -- grouping by attraction
pointer.execute("SELECT attraction, COUNT(*) as c FROM checkin WHERE attraction IN (SELECT AttractionID FROM attraction where LOWER(Category) LIKE '%ride%') GROUP BY attraction;")
list1 = pointer.fetchall()
visited_best = (0, 0)
ridetime_best = (0, math.inf)
both_best = (0, 0, math.inf)
for (attraction_id,count) in list1:
    #getting duration for calculation from checkin for required attractions
    pointer.execute("SELECT duration FROM checkin where attraction="+str(attraction_id)+";")
    list2 = pointer.fetchall()
    total_duration = 0
    i = 0
    for d in list2:
        try:
            duration = d[0].split(':')
            total_duration += (int(duration[0])*3600) + (int(duration[1])*60) +(int(duration[2]))
            i += 1
        except:
            continue
    avg_duration = total_duration / i
    if count > visited_best[1]:
        visited_best = (attraction_id, count)
    if avg_duration < ridetime_best[1]:
        ridetime_best = (attraction_id, avg_duration)
    if count > both_best[1] and avg_duration < both_best[2]:
        both_best = (attraction_id, count, avg_duration)
#getting name of attractions where attraction ID are same as the ones in best visited, best ridetime and best of both
pointer.execute("SELECT Name FROM attraction WHERE AttractionID="+str(visited_best[0])+" OR AttractionID="+str(ridetime_best[0])+" OR AttractionID="+str(both_best[0])+";")
names = pointer.fetchall()
name_list = [name[0] for name in names]
print(name_list)


['Wrightiraptor Mountain', 'Atmosfear', 'Dykesadactyl Thrill']
